In [10]:
import requests
from bs4 import BeautifulSoup
import os, sys
from datetime import datetime
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Km19982016",
    port="8081",
    database="testdb"
)

mycursor = mydb.cursor()

sublinks = ["/list/ls021409819/","/list/ls051600015/?st_dt=&mode=detail&sort=release_date,desc&page=1","/search/title?languages=hi&title_type=tv_series&page=1&sort=release_date,desc&view=advanced&ref_=adv_nxt"]
baselink = "https://www.imdb.com"


mp={"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"july":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}


def insert_to_db(data):
    sqlformulains = "INSERT INTO appusers (user_id,tvseries) VALUES (%s,%s)"
    mycursor.execute(sqlformulains,data)
    mydb.commit()
    return

def delete_from_db(uid):
    sqlformuladel ="DELETE FROM appusers WHERE user_id = "
    sqlformuladel += '"'
    sqlformuladel += uid
    sqlformuladel += '"'
    mycursor.execute(sqlformuladel)
    sqlformuladel ="DELETE FROM linker WHERE user_id = "
    sqlformuladel += '"'
    sqlformuladel += uid
    sqlformuladel += '"'
    mycursor.execute(sqlformuladel)
    mydb.commit()
    return

def get_rows():
    mycursor.execute("SELECT * FROM appusers")
    ans = mycursor.fetchall()
    n = len(ans)
    return n

def takeuserdata():
    print("Are you a new user?y/n")
    c = input()
    if(c=='y'):
        print("Enter your email-id:")
        mailid = input()
        print("Enter your preferences:")
        likes = input().strip()
        likes = likes.split(",")
        count = get_rows()
        count+=1
        sqlformulains = "INSERT INTO linker (user_id,email) VALUES (%s,%s)"
        data=list()
        data.insert(0,count)
        data.insert(1,mailid)
        mycursor.execute(sqlformulains,data)
        mydb.commit()
        for item in likes:
            val=list()
            val.insert(0,count)
            val.insert(1,str(item))
            val=tuple(val)
            insert_to_db(val)
        regusers[mailid]=countusers
    else:
        return
    return

def getmon(month):
    month = month.lower()
    month = month.replace(".","")
    num = mp[month]
    return num

def getserieslink(baselink,s,seriesname):
    serieslink=""
    while 1==1 :
        #print(baselink+s)
        r = requests.get(baselink+s)
        c=r.content
        soup = BeautifulSoup(c,"html.parser")
        try:
            nextall = soup.find_all("div",{"class":"lister-item mode-detail"})
            for item in nextall:
                next1 = item.find_all("h3",{"class":"lister-item-header"})[0]
                temp=next1.find("a").text
                #print(temp.lower())
                #print(temp.lower())
                if(temp.lower() == seriesname.lower()):
                    href = next1.find_all("a")[0]
                    href = href["href"]
                    serieslink=href
            if(len(serieslink)>0):
                break
            else:
                
                    nextpage = soup.find("div",{"class":"list-pagination"})
                    #print(nextpage)
                    nextpage = nextpage.find_all("a")[1]
                    #print(nextpage["href"])
                    if(nextpage["href"] != "#"):
                        s=nextpage["href"]
                    else:
                        break
               
                    
        except:
            nextall = soup.find_all("div",{"class":"lister-item mode-advanced"})
            for item in nextall:
                next1 = item.find_all("h3",{"class":"lister-item-header"})[0]
                temp=next1.find("a").text
                #print(temp.lower())
                #print(temp.lower())
                if(temp.lower() == seriesname.lower()):
                    href = next1.find_all("a")[0]
                    href = href["href"]
                    serieslink=href
                    break
            if(len(serieslink)>0):
                break
            else:
                nextpage = soup.find("div",{"class":"nav"})
                #print(nextpage)
                try:
                    nextpage = nextpage.find_all("a")[2]
                    #print(nextpage)
                    s=nextpage["href"]
                    s="/search/title"+s
                except:
                    break
    return serieslink


def getseriesdetails(link):
    details=""
    r = requests.get(link)
    c=r.content
    soup = BeautifulSoup(c,"html.parser")
    nextall = soup.find("div",{"class":"seasons-and-year-nav"})
    #print(nextall)
    nextall = nextall.find_all("a")
    for item in nextall:
        try:
            a = int(item.text)
            if (a>1000) :
                currentYear = datetime.now().year
                #print(currentYear)
                #print(a)
                if currentYear < a :
                    details="The next season begins in "
                    details+=str(a)
                    details+="."
                elif currentYear > a :
                    details="The show has finished streaming all its episodes.We will keep you updated!"
                else:
                    temp=nextall[0]
                    seasonlink = temp["href"]
                    #print(baselink+seasonlink)
                    rr = requests.get(baselink+seasonlink)
                    cc=rr.content
                    soupp = BeautifulSoup(cc,"html.parser")
                    nextalll = soupp.find_all("div",{"class":"airdate"})
                    #print(nextalll)
                    #print(len(nextalll[-1].text))
                    #print(nextalll[-2].text)
                    latestmon=datetime.now().month
                    latestday=datetime.now().day
                    for each in nextalll:
                        airtime = each.text.split()
                        try:
                            airtime[1]=getmon(airtime[1])
                            airtime[0]=int(airtime[0])
                            #print(airtime)
                            #print(datetime.now().month)
                            #print(datetime.now().day)
                            #print(airtime[1]==datetime.now().month)
                            #print(airtime[0]>=datetime.now().day)
                            if(airtime[1]==datetime.now().month):
                                if(airtime[0]>=datetime.now().day):
                                    latestday=airtime[0]
                                    latestmon=airtime[1]
                                    break
                                else:
                                    latestday=airtime[0]
                            elif(airtime[1]>datetime.now().month):
                                latestday=airtime[0]
                                latestmon=airtime[1]
                                break
                            else:
                                latestday=airtime[0]
                                latestmon=airtime[1]
                        except:
                            pass
                    try:
                        tempmon=latestmon
                        tempday=latestday
                        if latestmon in range(1,10):
                            latestmon = "0"+str(latestmon)
                        if latestday in range(1,10):
                            latestday = "0"+str(latestday)                        
                        ans=str(currentYear)+"-"+str(latestmon)+"-"+str(latestday)
                        if(tempmon<datetime.now().month or (tempmon==datetime.now().month and tempday<datetime.now().day)):
                            details="The show has finished streaming all its episodes.The last episode was aired on ";
                            details+=ans;
                        else:
                            details="Next episode airs on "+ans
                    except:
                        details="No Details Available"
        
                break
            else:
                pass
        except:
            pass
    return details
    

takeuserdata()

mycursor.execute("SELECT * FROM linker")
ans = mycursor.fetchall()
mycursor.execute("SELECT * FROM appusers")
allusers = mycursor.fetchall()
for item in ans:
    userid = item[0]
    message=""
    for users in allusers:
        if users[0]==userid:
            seriesname=users[1]
            temp=""
            for links in sublinks:
                serieslink = getserieslink(baselink,links,seriesname)
                #print(serieslink)
                if(len(serieslink)>0):
                    serieslink = baselink+serieslink
                    seriesdetails = getseriesdetails(serieslink)
                    temp=seriesdetails
                    break
            if(len(temp)==0):
                message+="TV Series: " + seriesname +"\n" + "Status: "
                temp="No Details Available"
                message+=temp
            else:
                message+="TV Series: " + seriesname +"\n" + "Status: "
                message+=temp
            message+="\n\n"
    print(message)

mycursor.close()
mydb.close()


Are you a new user?y/n
n
TV Series: sacred games
Status: The next season begins in 2019.
TV Series: game of thrones
Status: The next season begins in 2019.

